In [1]:
from model5LHzc import *
from data_1ch import *

vid="v029" #version id should match the file number 
# Ran on DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=3
learning_rate=1e-3

train_gen = trainGenerator(batch_size,'mass_seg_05/train05','mass_seg','mass_seg_labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_05/valid05','mass_seg','mass_seg_labels',data_gen_args,save_to_dir = None)

# 300 images are used for training, 52 images for validating, and 60 images for testing
train_steps = 300//batch_size
valid_steps = 52//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=2.5*train_steps
num_epochs=100
num_top_filter=24

model = unet(learning_rate, loss, num_top_filter)

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files05/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files05/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 300 images belonging to 1 classes.
Found 300 images belonging to 1 classes.
Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.8526 - acc: 0.9163 - dice_coef: 0.1474 - iou: 0.0853Found 52 images belonging to 1 classes.
Found 52 images belonging to 1 classes.
250/250 [==============================] - 95s 327ms/step - loss: 0.8526 - acc: 0.9163 - dice_coef: 0.1474 - iou: 0.0853 - val_loss: 0.8833 - val_acc: 0.8677 - val_dice_coef: 0.1167 - val_iou: 0.0681

Epoch 00001: saving model to files05\unet_mass_seg_v029.hdf5
Epoch 2/100
250/250 [==============================] - 83s 333ms/step - loss: 0.7519 - acc: 0.9655 - dice_coef: 0.2481 - iou: 0.1516 - val_loss: 0.7500 - val_acc: 0.9679 - val_dice_coef: 0.2500 - val_iou: 0.1551

Epoch 00002: saving model to files05\unet_mass_seg_v029.hdf5
Epoch 3/100
250/250 [==============================] - 81s 325ms/step - loss: 0.7325 - acc: 0.9708 - dice_coef: 0.2675 - iou: 0.1657 - val_loss: 0.7638 - val_acc: 0.9712 - val_d

### validate your model and save predicted results

In [3]:
validGene = testGenerator("mass_seg_05/valid05/pred")
model.load_weights("files05/unet_mass_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,52,verbose=1)
saveResult("mass_seg_05/valid05/pred",results,vid)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


52/52 [==============================] - 5s 60ms/step


D:\research\breastCancer\mass_seg\codes\data_1ch.py:125: UserWarning: mass_seg_05/valid05/pred\0_predict_v029.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data_1ch.py:125: UserWarning: mass_seg_05/valid05/pred\1_predict_v029.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data_1ch.py:125: UserWarning: mass_seg_05/valid05/pred\3_predict_v029.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data_1ch.py:125: UserWarning: mass_seg_05/valid05/pred\4_predict_v029.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data_1ch.py:125: UserWarning: mass_seg_05/valid05/pred\9_predict_v029.png is a low contrast image
  io.imsave(os.path.join(save_path